In [14]:
import tensorflow as tf
import numpy as np
from tensorflow.python.framework import ops

ops.reset_default_graph()

# ---------------------------------------------------|
# -------------------1D-data-------------------------|
# ---------------------------------------------------|

# Create graph session 
sess = tf.Session()
# Generate 1D data
data_size = 25
data_1d = np.random.normal(size=data_size)
print(data_1d)

[-0.26839399  2.60158466 -1.24131294 -0.45502629 -1.77953067  0.13782581
  0.22095009 -1.03822071  0.76556819  0.43275458  1.52816032 -1.94326924
  0.05509926  2.11127391  0.61317289 -0.07634732 -1.16257783  0.63697826
 -0.32768125  1.20707562 -0.98744165  0.84372634 -0.22266482  0.57540839
  0.77225308]


In [31]:
# Placeholder
x_input_1d = tf.placeholder(shape=[data_size],dtype=tf.float32)
# Create filter for convolution.
conv_filter_1 = tf.Variable(tf.random_normal(shape=[1,5,1,1]))

# --------Convolution--------
# input shape = [batch, in_height, in_width, in_channels]
# filter shape = [filter_height, filter_width, in_channels, out_channels]
def conv_layers_1d(input_1d,my_filter):
    input_2d = tf.expand_dims(input_1d,0)
    input_3d = tf.expand_dims(input_2d,0)
    input_4d = tf.expand_dims(input_3d,3)
    #input_4d like (1,1,input_1d,1)
    convolution_output = tf.nn.conv2d(input_4d,filter=my_filter,strides=[1,1,1,1],padding='VALID')
    #print(convolution_output) #Tensor("Conv2D_4:0", shape=(1, 1, 21, 1), dtype=float32)
    conv_output_1d = tf.squeeze(convolution_output)
#     print(conv_output_1) #Tensor("Squeeze:0", shape=(21,), dtype=float32)
    return conv_output_1d
# Create convolution layer
conv_output_1 = conv_layers_1d(x_input_1d,conv_filter_1)
# --------Activation Function--------
def activation(input_1d):
    return tf.nn.relu(input_1d)

# Create activation layer
activation_output_1 = activation(conv_output_1)
# --------Max Pool--------
'''
max pooling是CNN当中的最大值池化操作，其实用法和卷积很类似

有些地方可以从卷积去参考【TensorFlow】tf.nn.conv2d是怎样实现卷积的？ 

tf.nn.max_pool(value, ksize, strides, padding, name=None)
参数是四个，和卷积很类似：

第一个参数value：需要池化的输入，一般池化层接在卷积层后面，所以输入通常是feature map，依然是[batch, height, width, channels]这样的shape

第二个参数ksize：池化窗口的大小，取一个四维向量，一般是[1, height, width, 1]，因为我们不想在batch和channels上做池化，所以这两个维度设为了1

第三个参数strides：和卷积类似，窗口在每一个维度上滑动的步长，一般也是[1, stride,stride, 1]

第四个参数padding：和卷积类似，可以取'VALID' 或者'SAME'

返回一个Tensor，类型不变，shape仍然是[batch, height, width, channels]这种形式

'''
def max_pool(input_1d,width):
    # Just like 'conv2d()' above, max_pool() works with 4D arrays.
    # [batch_size=1, width=1, height=num_input, channels=1]
    input_2d = tf.expand_dims(input_1d,0)
    input_3d = tf.expand_dims(input_2d,0)
    input_4d = tf.expand_dims(input_3d,3)
    print(input_4d,width) #Tensor("ExpandDims_19:0", shape=(1, 1, 21, 1), dtype=float32) 5
    pool_output = tf.nn.max_pool(input_4d,ksize=[1,1,width,1],strides=[1,1,1,1],padding='VALID')
    print(pool_output)#Tensor("MaxPool_2:0", shape=(1, 1, 17, 1), dtype=float32)
    # Get rid of extra dimensions
    pool_output_1d = tf.squeeze(pool_output)
    print(pool_output_1d)#Tensor("Squeeze_9:0", shape=(17,), dtype=float32)
    return pool_output_1d

maxpool_output_1 = max_pool(activation_output_1,width=5)
# help(tf.nn.max_pool)
# --------Fully Connected--------
def fully_connected(input_layer,num_outputs):
    # First we find the needed shape of the multiplication weight matrix:
    # The dimension will be (length of input) by (num_outputs)
    # tf.pack modified to tf.stack
    print("full_connected")
    print(input_layer.get_shape())#Tensor("Squeeze_25:0", shape=(17,), dtype=float32)
    print(tf.shape(input_layer))#Tensor("Shape_7:0", shape=(1,), dtype=int32)
    print(tf.stack([tf.shape(input_layer),[num_outputs]]))#Tensor("stack_9:0", shape=(2, 1), dtype=int32)
    weight_shape = tf.squeeze(tf.stack([tf.shape(input_layer),[num_outputs]]))
    print(weight_shape)#Tensor("Squeeze_18:0", shape=(2,), dtype=int32)
    # Initialize such weight
    weight = tf.random_normal(weight_shape,stddev=0.1)
    print(weight_shape)#Tensor("Squeeze_18:0", shape=(2,), dtype=int32)
    # Initialize the bias
    bias = tf.random_normal(shape=[num_outputs])
    print(bias)#Tensor("random_normal_14:0", shape=(5,), dtype=float32)
    # Make the 1D input array into a 2D array for matrix multiplication
    input_layer_2d = tf.expand_dims(input_layer,0)
    # Perform the matrix multiplication and add the bias
    full_output = tf.add(tf.matmul(input_layer_2d,weight),bias)
    print(full_output)#Tensor("Add_4:0", shape=(1, 5), dtype=float32)
    # Get rid of extra dimensions
    full_output_1d = tf.squeeze(full_output)
    print(full_output_1d)#Tensor("Squeeze_23:0", shape=(5,), dtype=float32)
    return full_output_1d

full_output_1 = fully_connected(maxpool_output_1,num_outputs=5)
# Run graph
# Initialize Variables
init = tf.global_variables_initializer()
sess.run(init)
feed_dict = {x_input_1d: data_1d}
help(tf.shape)


Tensor("ExpandDims_96:0", shape=(1, 1, 21, 1), dtype=float32) 5
Tensor("MaxPool_13:0", shape=(1, 1, 17, 1), dtype=float32)
Tensor("Squeeze_53:0", shape=(17,), dtype=float32)
full_connected
(17,)
Tensor("Shape_23:0", shape=(1,), dtype=int32)
Tensor("stack_17:0", shape=(2, 1), dtype=int32)
Tensor("Squeeze_54:0", shape=(2,), dtype=int32)
Tensor("Squeeze_54:0", shape=(2,), dtype=int32)
Tensor("random_normal_41:0", shape=(5,), dtype=float32)
Tensor("Add_13:0", shape=(1, 5), dtype=float32)
Tensor("Squeeze_55:0", shape=(5,), dtype=float32)
Help on function shape in module tensorflow.python.ops.array_ops:

shape(input, name=None, out_type=tf.int32)
    Returns the shape of a tensor.
    
    This operation returns a 1-D integer tensor representing the shape of `input`.
    
    For example:
    
    ```python
    # 't' is [[[1, 1, 1], [2, 2, 2]], [[3, 3, 3], [4, 4, 4]]]
    shape(t) ==> [2, 2, 3]
    ```
    
    Args:
      input: A `Tensor` or `SparseTensor`.
      name: A name for the opera

In [5]:
# Convolution Output
print(x_input_1d)
print(conv_filter_1,sess.run(conv_filter_1))
print(conv_output_1)
print("Convolution Output--->",sess.run(conv_output_1,feed_dict=feed_dict))

# test conv2d function
input = tf.Variable(tf.random_normal([10,5,5,5])) # shape = [batch, in_height, in_width, in_channels]
filter = tf.Variable(tf.random_normal([3,3,5,7])) # shape = [filter_height, filter_width, in_channels, out_channels]
op = tf.nn.conv2d(input, filter, strides=[1, 1, 1, 1], padding='VALID') #output shape = [batch, height, width, out_channels]
print(op)

Tensor("Placeholder:0", shape=(25,), dtype=float32)
<tf.Variable 'Variable:0' shape=(1, 5, 1, 1) dtype=float32_ref> [[[[-0.26380596]]

  [[-0.09360479]]

  [[-1.03008616]]

  [[-1.10657895]]

  [[ 1.61247182]]]]
Tensor("Squeeze:0", shape=(21,), dtype=float32)
Convolution Output---> [-5.25032902 -0.76354825  2.38797832 -2.96033144  1.4671458   2.20014429
 -4.3531661   1.08731711  1.35556352 -1.676211    0.55022949  0.87963533
 -1.12203455 -2.56535292  1.1218183  -0.49584508  1.55208647  4.10489845
  1.92178488 -5.42443514  2.56427956]
Tensor("Conv2D_1:0", shape=(10, 3, 3, 7), dtype=float32)


In [6]:
# Activation Output
print(sess.run(activation_output_1, feed_dict=feed_dict))

# Max Pool Output
print(sess.run(maxpool_output_1, feed_dict=feed_dict))

# Fully Connected Output
print(sess.run(full_output_1, feed_dict=feed_dict))

[ 0.          0.          2.38797832  0.          1.4671458   2.20014429
  0.          1.08731711  1.35556352  0.          0.55022949  0.87963533
  0.          0.          1.1218183   0.          1.55208647  4.10489845
  1.92178488  0.          2.56427956]
[ 2.38797832  2.38797832  2.38797832  2.20014429  2.20014429  2.20014429
  1.35556352  1.35556352  1.35556352  0.87963533  1.1218183   1.1218183
  1.55208647  4.10489845  4.10489845  4.10489845  4.10489845]
[ 3.42370319  0.64030182 -0.60373384 -0.64634222 -3.40978742]


In [45]:
# ---------------------------------------------------|
# -------------------2D-data-------------------------|
# ---------------------------------------------------|

# Reset Graph
ops.reset_default_graph()
sess = tf.Session()

# Generate 2D data
data_size = [10, 10]
data_2d = np.random.normal(size=data_size)

# --------Placeholder--------
x_input_2d = tf.placeholder(dtype=tf.float32, shape=data_size)


# Convolution
def conv_layer_2d(input_2d, my_filter):
    # Tensorflow's 'conv2d()' function only works with 4D arrays:
    # [batch#, width, height, channels], we have 1 batch, and
    # 1 channel, but we do have width AND height this time.
    # So next we create the 4D array by inserting dimension 1's.
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    # Note the stride difference below!
    print(input_4d) #Tensor("ExpandDims_1:0", shape=(1, 10, 10, 1), dtype=float32)
    print(my_filter)#<tf.Variable 'Variable:0' shape=(2, 2, 1, 1) dtype=float32_ref>
    convolution_output = tf.nn.conv2d(input_4d, filter=my_filter, strides=[1, 2, 2, 1], padding="VALID")
    print(convolution_output) #Tensor("Conv2D:0", shape=(1, 5, 5, 1), dtype=float32)
    # Get rid of unnecessary dimensions
    conv_output_2d = tf.squeeze(convolution_output)
    return conv_output_2d


# Create Convolutional Filter
my_filter = tf.Variable(tf.random_normal(shape=[2, 2, 1, 1]))
# Create Convolutional Layer
my_convolution_output = conv_layer_2d(x_input_2d, my_filter)


# --------Activation--------
def activation(input_1d):
    return tf.nn.relu(input_1d)


# Create Activation Layer
my_activation_output = activation(my_convolution_output)


# --------Max Pool--------
def max_pool(input_2d, width, height):
    # Just like 'conv2d()' above, max_pool() works with 4D arrays.
    # [batch_size=1, width=given, height=given, channels=1]
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    # Perform the max pooling with strides = [1,1,1,1]
    # If we wanted to increase the stride on our data dimension, say by
    # a factor of '2', we put strides = [1, 2, 2, 1]
    pool_output = tf.nn.max_pool(input_4d, ksize=[1, height, width, 1],
                                 strides=[1, 1, 1, 1],
                                 padding='VALID')
    # Get rid of unnecessary dimensions
    pool_output_2d = tf.squeeze(pool_output)
    return pool_output_2d


# Create Max-Pool Layer
my_maxpool_output = max_pool(my_activation_output, width=2, height=2)


# --------Fully Connected--------
def fully_connected(input_layer, num_outputs):
    print("fully_2d")
    # In order to connect our whole W byH 2d array, we first flatten it out to
    # a W times H 1D array.
    print(input_layer)#Tensor("Squeeze_1:0", shape=(4, 4), dtype=float32)
    flat_input = tf.reshape(input_layer, [-1])
    print(flat_input)#Tensor("Reshape:0", shape=(16,), dtype=float32)
    # We then find out how long it is, and create an array for the shape of
    # the multiplication weight = (WxH) by (num_outputs)
    # tf.pack modified to tf.stack
    print(tf.shape(flat_input))#Tensor("Shape:0", shape=(1,), dtype=int32)
    weight_shape = tf.squeeze(tf.stack([tf.shape(flat_input), [num_outputs]]))
    print(weight_shape)#Tensor("Squeeze_2:0", shape=(2,), dtype=int32)
    # Initialize the weight
    weight = tf.random_normal(weight_shape, stddev=0.1)
    # Initialize the bias
    bias = tf.random_normal(shape=[num_outputs])
    print(bias)#Tensor("random_normal_2:0", shape=(5,), dtype=float32)
    # Now make the flat 1D array into a 2D array for multiplication
    input_2d = tf.expand_dims(flat_input, 0)
    # Multiply and add the bias
    full_output = tf.add(tf.matmul(input_2d, weight), bias)
    # Get rid of extra dimension
    full_output_2d = tf.squeeze(full_output)
    return full_output_2d

# Create Fully Connected Layer
my_full_output = fully_connected(my_maxpool_output, 5)


Tensor("ExpandDims_1:0", shape=(1, 10, 10, 1), dtype=float32)
<tf.Variable 'Variable:0' shape=(2, 2, 1, 1) dtype=float32_ref>
Tensor("Conv2D:0", shape=(1, 5, 5, 1), dtype=float32)
fully_2d
Tensor("Squeeze_1:0", shape=(4, 4), dtype=float32)
Tensor("Reshape:0", shape=(16,), dtype=float32)
Tensor("Shape:0", shape=(1,), dtype=int32)
Tensor("Squeeze_2:0", shape=(2,), dtype=int32)
Tensor("random_normal_2:0", shape=(5,), dtype=float32)


In [46]:
# Run graph
# Initialize Variables
init = tf.initialize_all_variables()
sess.run(init)

feed_dict = {x_input_2d: data_2d}

# Convolution Output
print(sess.run(my_convolution_output, feed_dict=feed_dict))

# Activation Output
print(sess.run(my_activation_output, feed_dict=feed_dict))

# Max Pool Output
print(sess.run(my_maxpool_output, feed_dict=feed_dict))

# Fully Connected Output
print(sess.run(my_full_output, feed_dict=feed_dict))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
[[-0.5984751  -0.44325101  2.00933743 -1.11890626 -1.61312366]
 [-1.35967588 -0.46474075 -0.63701457 -0.72974437 -0.14482903]
 [-1.08951175 -0.78064382 -0.92946291 -0.9610787   1.92230773]
 [-1.56241274  0.97630847 -1.3101933  -0.33032238  0.22170272]
 [-0.62391078  0.42604107 -1.31330931  1.21313882  0.62701559]]
[[ 0.          0.          2.00933743  0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          1.92230773]
 [ 0.          0.97630847  0.          0.          0.22170272]
 [ 0.          0.42604107  0.          1.21313882  0.62701559]]
[[ 0.          2.00933743  2.00933743  0.        ]
 [ 0.          0.          0.          1.92230773]
 [ 0.97630847  0.97630847  0.          1.92230773]
 [ 0.97630847  0.97630847  1.21313882  1.21313882]]
[ 0.53567219 -0.48650131 -0.62142676  0.13522327 -0.38255793]
